In [1]:
import numpy as np
import pandas as pd

In [2]:
# wir laden unsere ratings und unsere genres
ratings = pd.read_csv('workshop_user_ratings.csv')
genres = pd.read_csv('workshop_movie_genres.csv')

In [3]:
# dieser Datensatz ist eine bearbeitete Kopie der Tabelle auf dem Google Drive. Was hat sich geändert? Warum?
ratings

,user_id,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,user1,4,0,5,5,0,3,5,5,3,...,3,2,3,5,5,4,2,1,2,4
1,user2,4,0,5,4,3,3,3,3,0,...,5,3,4,3,4,3,2,1,5,3
2,user3,0,0,4,3,0,5,4,4,3,...,2,4,3,5,5,2,2,0,4,4
3,user4,1,0,5,2,0,0,4,4,5,...,0,0,4,0,0,3,4,0,0,4
4,user5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,5
5,user6,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
6,user7,0,0,5,3,0,2,0,5,5,...,4,5,4,2,0,3,3,0,0,5
7,user8,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,3,3,4,2
8,user9,0,1,5,4,0,3,3,4,5,...,5,4,4,0,0,1,1,1,4,4
9,user10,5,0,4,4,0,0,0,2,5,...,3,5,0,5,5,0,0,0,0,4


Die Filmtitel stehen nicht drin. Diese sind durch ihre ID mit den Filmtiteln im Genres-Datensatz verknüpft.

In [5]:
# Wir brauchen eine Matrix, die nur die Ratings enthält
# Jede Zeile ist ein User und jede Spalte ist ein Film
rating_cols = list(genres.movie_id)
ratings_matrix = ratings[rating_cols].as_matrix()

Wir wollen nun die Cosine Similarity zwischen den Usern berechnen, nach der Formel

$$sim_{cos}(u_1, u_2) = \frac{u_1u_2}{|u_1||u_2|}$$

$|u_1|$ ist hier die Euklidische Norm.

Natürlich ist das längst für uns in sklearn implementiert.

In [8]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(ratings_matrix, metric='cosine')

Nun wollen wir vorhersagen, wie sehr unseren Usern andere Filme gefallen könnten. Die Wahrscheinlichkeit, dass User k Film j mag ist:

$$p(k,j) = \bar x_k + \frac {\sum_u sim_{cos}(u_k, u_i)(x_{i,j} - \bar x_i)}{\sum_u| sim_{cos}(u_k, u_i)|}$$

$\bar x_k$: durchschnittliches Rating von user k
$x_{i,j}$: Rating von User i über Film j

In [10]:
# Wir setzen die obere Formel matrixweise um:

# berechne durchschnittliches Rating
mean_user_rating = ratings_matrix.mean(axis=1)

# berechne die Differenz von jedem Rating von jedem User zu seinem durchschnittlichen Rating
ratings_diff = (ratings_matrix - mean_user_rating[:, np.newaxis])

# berechne den Zähler des Bruchs
zaehler = user_similarity.dot(ratings_diff)

# berechne den Nenner
nenner = np.array([np.abs(user_similarity).sum(axis=1)]).T

# zusammenfügen
prediction = mean_user_rating[:, np.newaxis] +  zaehler / nenner

In [17]:
# Schließlich schreiben wir das wieder in ein Dataframe, damit wir uns die Ergebnisse anschauen können
user_prediction = pd.DataFrame(prediction, index=ratings.user_id, columns=genres.title)

In [36]:
user_prediction.loc['user1']

title
Mad Max: Fury Road             3.224467
Fast and Furious 7             2.583572
Fight Club                     4.672461
The Dark Knight                3.714077
Vanilla Sky                    2.349372
Star Wars Episode IV           2.925533
Rocky                          3.393908
Pretty Woman                   3.391887
Django Unchained               3.716656
Almost Famous                  2.154032
Back to the Future             3.397978
Clueless                       2.666834
The Big Lebowski               2.610767
My Big Fat Greek Wedding       2.522350
Saving Private Ryan            2.960508
Kill Bill                      4.227388
Guardians of the Galaxy        3.092763
Titanic                        3.778152
Deadpool                       3.189072
Ten Things I Hate About You    2.695000
Bridget Jones's Diary          2.641535
Twilight                       2.783209
Wonder Woman                   2.782844
La La Land                     3.509726
The Matrix                     5.0

Übung: Statt Userähnlichkeit könnten wir auch die Filmähnlichkeit betrachten. Die Wahrscheinlichkeit, dass User k Film j mag, ergibt sich aus:

$$p(k, j) = \frac {\sum_m sim_{cos}(m_i, m_j)x_{k,i}}{\sum_m | sim_{cos}(m_i, m_j)|}$$

$x_{k,i}$: Rating von User k über Film i

$m_k$: Ratingvektor von movie k

In [ ]:
# Übung: Statt Userähnlichkeit könnten wir auch die Filmähnlichkeit betrachten.
# Berechne dafür zunächst cosine distances zwischen den Filmen (die Matrix sollte 25*25 sein)
movie_similarity = 

In [ ]:
# Nun berechnen wir den dot product der rating matrix und der movie similarity matrix
zaehler = 

# Das Ganze teilen wir dann durch die Summe aller movie similarities (siehe oben...)

nenner = 

In [ ]:
item_based_prediction = zaehler / nenner
item_prediction = pd.DataFrame(item_pred, index=ratings.user_id, columns=genres.title)

In [26]:
# Hier brauchen wir nicht mehr mit den durchschnittlichen Userwertungen zu normieren, da wir nur noch user k betrachten
item_pred = ratings_matrix.dot(movie_similarity) / np.array([np.abs(movie_similarity).sum(axis=1)])

In [32]:
item_prediction = pd.DataFrame(item_pred, index=ratings.user_id, columns=genres.title)

In [35]:
item_prediction.loc['user1']

title
Mad Max: Fury Road             2.682291
Fast and Furious 7             3.212084
Fight Club                     2.622090
The Dark Knight                2.449252
Vanilla Sky                    3.357630
Star Wars Episode IV           2.740546
Rocky                          2.681780
Pretty Woman                   2.761920
Django Unchained               2.784424
Almost Famous                  2.792345
Back to the Future             2.611174
Clueless                       2.824278
The Big Lebowski               2.654078
My Big Fat Greek Wedding       3.329470
Saving Private Ryan            3.101312
Kill Bill                      2.912147
Guardians of the Galaxy        2.721952
Titanic                        2.893612
Deadpool                       2.646994
Ten Things I Hate About You    2.689469
Bridget Jones's Diary          2.730925
Twilight                       2.984089
Wonder Woman                   3.159246
La La Land                     3.083529
The Matrix                     2.7

# Bonus
Falls noch Zeit da ist... Wir haben ja auch noch andere Daten über User erhoben.

In [37]:
data = pd.read_csv('workshop_user_info.csv')
data

,frage,user1,user2,user3,user4,user5,user6,user7,user8,user9,user10,user11,user12,user13,user14,user15,user16,user17,user18,user19
0,Alter,34,30,26,25,26,27,25,21,27,24,39,23,35,42,37,28,29,30,47
1,Geschlecht,1,1,1,1,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0
2,Koriander,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0


In [ ]:
# Können wir aus den Filmpräferenzen das Alter und das Geschlecht der Teilnehmer vorhersagen? Ihre Koriandervorlieben?
# Sollten wir Alter und Geschlecht in die Ähnlichkeitsmatrix einfließen lassen? Wie?
# Überlegt euch zunächst, in welcher Form ihr die Daten für euer Experiment braucht. Dann könnt ihr gerne um Hilfe fragen,
# um die Daten zusammenzufügen. Oder einen Shortcut nehmen und Excel o. Ä. benutzen, um das schnell selbst zu
# machen :)